In [2]:
!pip install transformers==4.51.3 numpy tqdm accelerate datasets scikit-learn -q
print('done')


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
done


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from pathlib import Path
import json
from tqdm import tqdm
import os
from accelerate import init_empty_weights
from functools import partial
import random
from torch.utils.data import DataLoader
from datasets import load_dataset
from torch.linalg import svd, norm

class SVDQwen3MoeSparseMoeBlock(nn.Module):
    def __init__(self, config,dtype= torch.bfloat16):
        super().__init__()
        self.num_experts = config.num_experts
        self.top_k = config.num_experts_per_tok
        self.norm_topk_prob = config.norm_topk_prob
        self.hidden_dim = config.hidden_size
        self.intermediate_dim = config.moe_intermediate_size
        # gating
        self.dtype = dtype
        
        self.gate = nn.Linear(config.hidden_size, config.num_experts, bias=False, dtype=self.dtype)
        # share v
        self.low_rank = int(self.intermediate_dim * self.hidden_dim * 0.3 / (self.intermediate_dim + self.hidden_dim))
        self.experts_v1_shared_gate = nn.Linear(self.hidden_dim, self.low_rank, bias=False, dtype=self.dtype)
        self.experts_v2_shared_up = nn.Linear(self.hidden_dim, self.low_rank, bias=False, dtype=self.dtype)
        self.experts_v3_shared_down = nn.Linear(self.intermediate_dim, self.low_rank, bias=False, dtype=self.dtype)
        self.beta = 0.9
        self.experts = nn.ModuleList(
            [moe_layer(config, False, self.experts_v1_shared_gate, self.experts_v2_shared_up, self.experts_v3_shared_down)  for _ in range(self.num_experts)])
        # share v false
        # self.experts = nn.ModuleList(
        #     [moe_layer(config, intermediate_size=config.moe_intermediate_size) for _ in range(self.num_experts)]
        # )

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        batch_size, sequence_length, hidden_dim = hidden_states.shape
        hidden_states = hidden_states.to(self.gate.weight.dtype)
        hidden_states = hidden_states.view(-1, hidden_dim)
        
        # Compute router logits and routing weights
        router_logits = self.gate(hidden_states)
        routing_weights = F.softmax(router_logits, dim=1, dtype=torch.float)
        routing_weights, selected_experts = torch.topk(routing_weights, self.top_k, dim=-1)
        
        # Dynamic skipping: if any expert weight < beta * top_expert_weight, skip it
        if hasattr(self, 'beta') and self.beta > 0:
            # Create mask for experts to skip (all except top-1)
            mask_skip = routing_weights[:, 1:] < self.beta * routing_weights[:, [0]]
            # Zero out weights for skipped experts
            routing_weights[:, 1:][mask_skip] = 0
        
        # Normalize routing weights (including after dynamic skipping)
        if self.norm_topk_prob:  # only diff with mixtral sparse moe block!
            routing_weights /= routing_weights.sum(dim=-1, keepdim=True)
        
        routing_weights = routing_weights.to(hidden_states.dtype)
        
        # Prepare output tensor
        final_hidden_states = torch.zeros(
            (batch_size * sequence_length, hidden_dim), 
            dtype=hidden_states.dtype, 
            device=hidden_states.device
        )
        
        # Build expert mask
        expert_mask = torch.nn.functional.one_hot(selected_experts, num_classes=self.num_experts).permute(2, 1, 0)
        
        # Apply dynamic skip mask to expert_mask
        if hasattr(self, 'beta') and self.beta > 0:
            # Zero out expert assignments for dynamically skipped experts
            for k_idx in range(1, self.top_k):
                expert_mask[:, k_idx, mask_skip[:, k_idx-1]] = 0
        
        # Find which experts are actually used (optimization for 128 experts)
        expert_hit = torch.greater(expert_mask.sum(dim=(-1, -2)), 0).nonzero().squeeze(-1)
        
        # Process each active expert
        for expert_idx in expert_hit:
            expert_layer = self.experts[expert_idx]
            idx, top_x = torch.where(expert_mask[expert_idx])
            
            if top_x.numel() == 0:  # Skip if no tokens assigned to this expert
                continue
                
            # Get input tokens for this expert
            current_state = hidden_states[top_x].reshape(-1, hidden_dim)
            
            # Apply expert and weight by routing weights
            current_hidden_states = expert_layer(current_state) * routing_weights[top_x, idx, None]
            
            # Accumulate results
            final_hidden_states.index_add_(0, top_x, current_hidden_states.to(hidden_states.dtype))
        
        # Reshape back to original dimensions
        final_hidden_states = final_hidden_states.reshape(batch_size, sequence_length, hidden_dim)
        
        return final_hidden_states, router_logits

    def SVD(self, modules, top_k_svd = 40,num_experts = 128):
        self.gate.weight = modules.gate.weight
        v_list = []
        for i in range(num_experts):
          if i % 10 == 0:
            print(f'gate_proj의 {i}/128번째 expert svd')
              
          u,s,v = torch.svd_lowrank(modules.experts[i].gate_proj.weight.float(), q=self.experts[0].low_rank)
          # v_list.append(v)
          # u_list의 평균값 구하기
          self.experts[i].v1.weight = nn.Parameter(v.T)
          # US_top = u[:, :top_k_svd] * torch.diag(s[:top_k_svd])
          US_top = u @ torch.diag(s)
          self.experts[i].us1.weight = nn.Parameter(US_top)
          del u
          del s
          del v
        # self.shared_u = v_list.average()
          if i % 20 == 0:
            torch.cuda.empty_cache()
        # self.shared_u = torch.stack(v_list, dim=0).mean(dim=0)


        v_list = []
        for i in range(num_experts):
          if i % 10 == 0:
            print(f'up proj의 {i}/128번째 svd')
          u,s,v = torch.svd_lowrank(modules.experts[i].up_proj.weight.float(), q=self.experts[0].low_rank)
          # v_list.append(v)
          self.experts[i].v2.weight = nn.Parameter(v.T)
          # US_top = u[:, :top_k_svd] * torch.diag(s[:top_k_svd])
          US_top = u @ torch.diag(s)
          self.experts[i].us2.weight = nn.Parameter(US_top)
          del u
          del s
          del v
          if i % 20 == 0:
            torch.cuda.empty_cache()            
        # self.shared_u = v_list.average()
        # self.shared_u = torch.stack(v_list, dim=0).mean(dim=0)


        v_list = []
        for i in range(num_experts):
          if i % 10 == 0:
            print(f'down_proj의 {i}/128번째 svd')
          u,s,v = torch.svd_lowrank(modules.experts[i].down_proj.weight.float(), q=self.experts[0].low_rank)
          # v_list.append(v)
          self.experts[i].v3.weight = nn.Parameter(v.T)
          # US_top = u[:, :top_k_svd] * torch.diag(s[:top_k_svd])
          US_top = u @ torch.diag(s)
          self.experts[i].us3.weight = nn.Parameter(US_top)
          del u
          del s
          del v
          if i % 20 == 0:
            torch.cuda.empty_cache()            
        # self.shared_u = v_list.average()
        # self.shared_u = torch.stack(v_list, dim=0).mean(dim=0)


class moe_layer(nn.Module):
  def __init__(self, config, share_v, shared_vgate, shared_vup, shared_vdown ):
      super().__init__()

      self.intermediate_dim = config.moe_intermediate_size #768
      self.hidden_dim = config.hidden_size #2048
      self.dtype = torch.bfloat16
      self.low_rank = int(self.intermediate_dim * self.hidden_dim * 0.3 / (self.intermediate_dim + self.hidden_dim))
      # self.delta_ratio = 0.3
      self.act_fn = F.relu #ACT2FN[config.hidden_act]
      # self.low_rank =
      if share_v == True:
        self.v1 = shared_vgate
        # self.experts_v1_shared_gate = nn.Linear(self.hidden_dim, self.low_rank, bias=False, dtype=torch.bfloat16)         
        self.us1 = nn.Linear(self.low_rank, self.hidden_dim, bias=False, dtype=torch.bfloat16)
        self.v2 = shared_vup
        self.us2 = nn.Linear(self.low_rank, self.hidden_dim, bias=False, dtype=torch.bfloat16)
        self.v3 = shared_vdown
        self.us3 = nn.Linear(self.low_rank, self.intermediate_dim, bias=False, dtype=torch.bfloat16)
      else:
        self.v1 = nn.Linear(self.hidden_dim, self.low_rank, bias=False, dtype=torch.bfloat16)
        self.us1 = nn.Linear(self.low_rank, self.hidden_dim, bias=False, dtype=torch.bfloat16)
        self.v2 = nn.Linear(self.hidden_dim, self.low_rank, bias=False, dtype=torch.bfloat16)
        self.us2 = nn.Linear(self.low_rank, self.hidden_dim, bias=False, dtype=torch.bfloat16)
        self.v3 = nn.Linear(self.intermediate_dim, self.low_rank, bias=False, dtype=torch.bfloat16)
        self.us3 = nn.Linear(self.low_rank, self.intermediate_dim, bias=False, dtype=torch.bfloat16)
      # self.low_rank = int(self.intermediate_dim * self.hidden_dim * self.delta_ratio / (self.intermediate_dim + self.hidden_dim))

  def forward(self,hidden_states):
      hidden_states = hidden_states.to(self.v1.weight.dtype)
      gate = self.us1(self.v1(hidden_states))
      up = self.us2(self.v2(hidden_states))
      x = self.us3(self.v3(self.act_fn(gate) * up))
      return x

#  (gate_proj): Linear(in_features=2048, out_features=768, bias=False)
#           (up_proj): Linear(in_features=2048, out_features=768, bias=False)
#           (down_proj): Linear(in_features=768, out_features=2048, bias=False)
#           (act_fn): SiLU()
        # down_proj = self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))
#mixtral        up = current_up_hidden_states + self.delta_u3(current_up_deltav_hidden_states)
                  # gate = current_gate_hidden_states + self.delta_u1(current_gate_deltav_hidden_states)
                  # self.Wmean2(self.act_fn(gate) * up) + self.delta_u2(self.delta_v2(self.act_fn(gate) * up))

class Qwen3MoeSparseMoeBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_experts = config.num_experts
        self.top_k = config.num_experts_per_tok
        self.norm_topk_prob = config.norm_topk_prob

        self.gate = nn.Linear(config.hidden_size, config.num_experts, bias=False)
        self.experts = nn.ModuleList(
            [Qwen3MoeMLP(config, intermediate_size=config.moe_intermediate_size) for _ in range(self.num_experts)]
        )
        # self.gate_proj
        # self.up_proj
        # self.down_proje

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        """ """
        batch_size, sequence_length, hidden_dim = hidden_states.shape
        hidden_states = hidden_states.view(-1, hidden_dim)
        # router_logits: (batch * sequence_length, n_experts)
        router_logits = self.gate(hidden_states)

        routing_weights = F.softmax(router_logits, dim=1, dtype=torch.float)
        routing_weights, selected_experts = torch.topk(routing_weights, self.top_k, dim=-1)
        if self.norm_topk_prob:  # only diff with mixtral sparse moe block!
            routing_weights /= routing_weights.sum(dim=-1, keepdim=True)
        # we cast back to the input dtype
        routing_weights = routing_weights.to(hidden_states.dtype)

        final_hidden_states = torch.zeros(
            (batch_size * sequence_length, hidden_dim), dtype=hidden_states.dtype, device=hidden_states.device
        )

        expert_mask = torch.nn.functional.one_hot(selected_experts, num_classes=self.num_experts).permute(2, 1, 0)

        # Loop over all available experts in the model and perform the computation on each expert
        expert_hit = torch.greater(expert_mask.sum(dim=(-1, -2)), 0).nonzero()
        for expert_idx in expert_hit:
            expert_layer = self.experts[expert_idx]
            idx, top_x = torch.where(expert_mask[expert_idx].squeeze(0))

            current_state = hidden_states[None, top_x].reshape(-1, hidden_dim)
            current_hidden_states = expert_layer(current_state) * routing_weights[top_x, idx, None]

            final_hidden_states.index_add_(0, top_x, current_hidden_states.to(hidden_states.dtype))
        final_hidden_states = final_hidden_states.reshape(batch_size, sequence_length, hidden_dim)
        return final_hidden_states, router_logits

    def merge_mlp_k_svd(
        self,
        layer_names: list[str] = ["gate_proj", "up_proj", "down_proj"],
        threshold=0.9,
        n_atoms=50,
        sparsity=5,
        n_iter=10,
        device="cpu"
    ):
        """
        Apply shared dictionary K-SVD to all projection layers of Qwen3MoeMLP experts.
        """

        # Collect weights and map expert-layer positions
        all_weights = {layer: [] for layer in layer_names}
        expert_map = {layer: [] for layer in layer_names}

        for expert_idx, expert in enumerate(self.experts):
            for layer in layer_names:
                if not hasattr(expert, layer):
                    raise AttributeError(f"Expert {expert_idx} missing layer '{layer}'")
                linear_layer = getattr(expert, layer)
                if not isinstance(linear_layer, nn.Linear):
                    continue  # Already compressed or non-linear layer

                weight = linear_layer.weight.detach().cpu().numpy()
                all_weights[layer].append(weight)
                expert_map[layer].append((expert_idx, linear_layer.weight.shape))

        # Run K-SVD + clustering per layer type
        compressed_results = {}
        for layer in layer_names:
            if len(all_weights[layer]) == 0:
                continue
            print(f"Compressing layer {layer} with {len(all_weights[layer])} experts.")
            # print(f'{all_weights[layer]}')
            # Apply clustering and K-SVD
            results = cluster_and_shared_ksvd(
                all_weights[layer],
                threshold=threshold,
                n_atoms=n_atoms,
                sparsity=sparsity,
                n_iter=n_iter
            )
            compressed_results[layer] = results

            # ---- layer별 D_shared 한 번만 생성 ----
            D_shared = nn.Parameter(torch.tensor(results[0]['D'], dtype=torch.float32, device=device))
            setattr(self, f"D_shared_{layer}", D_shared)

            # 각 expert별 X 하나만 생성
            for expert_idx, (exp_idx, weight_shape) in enumerate(expert_map[layer]):
                # results[0]['X_cluster']에서 expert별 slice 하나만 선택
                X = torch.tensor(results[0]['X_cluster'][:, expert_idx], dtype=torch.float32, device=device)
                shared_linear = SharedDictLinear(D_shared, X, bias=False).to(device)
                setattr(self.experts[exp_idx], layer, shared_linear)

        return compressed_results


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --------------------------
# 간단한 Qwen3MoeMLP 정의 (토이)
# --------------------------
class Qwen3MoeMLP(nn.Module):
    def __init__(self, config, intermediate_size):
        super().__init__()
        self.fc1 = nn.Linear(config.hidden_size, intermediate_size)
        self.fc2 = nn.Linear(intermediate_size, config.hidden_size)

    def forward(self, x):
        return F.relu(self.fc1(x)) @ self.fc2.weight.T  # weight를 사용하는 간단 예시

class Qwen3MoeMLP(nn.Module):
    def __init__(self, config, intermediate_size=None):
        super().__init__()
        self.config = config
        self.hidden_size = config.hidden_size
        self.intermediate_size = intermediate_size if intermediate_size is not None else config.intermediate_size
        self.gate_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=False)
        self.up_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=False)
        self.down_proj = nn.Linear(self.intermediate_size, self.hidden_size, bias=False)
        self.act_fn = F.relu #ACT2FN[config.hidden_act]

    def forward(self, x):
        print(self.gate_proj.weight.shape)
        print(self.up_proj.weight.shape)
        print(self.down_proj.weight.shape)
        down_proj = self.down_proj(self.act_fn(self.gate_proj(x)) * self.up_proj(x))
        return down_proj


In [5]:
class DummyConfig:
    def __init__(self):
        self.architectures = ["custommodel"]
        self.attention_bias = False
        self.attention_dropout = 0.0
        self.bos_token_id = 151643
        self.decoder_sparse_step = 1
        self.eos_token_id = 151645
        self.head_dim = 128
        self.hidden_act = "silu"
        self.hidden_size = 2048
        self.initializer_range = 0.02
        self.intermediate_size = 6144
        self.max_position_embeddings = 40960
        self.max_window_layers = 48
        self.mlp_only_layers = []
        self.model_type = "qwen3_moe"
        self.moe_intermediate_size = 768
        self.norm_topk_prob = True
        self.num_attention_heads = 32
        self.num_experts = 128
        self.num_experts_per_tok = 8
        self.num_hidden_layers = 48
        self.num_key_value_heads = 4
        self.output_router_logits = False
        self.rms_norm_eps = 1e-6
        self.rope_scaling = None
        self.rope_theta = 1000000.0
        self.router_aux_loss_coef = 0.001
        self.sliding_window = None
        self.tie_word_embeddings = False
        self.torch_dtype = "bfloat16"
        self.transformers_version = "4.51.0"
        self.use_cache = True
        self.use_sliding_window = False
        self.vocab_size = 151936

        # Qwen3MoeSparseMoeBlock 관련 필수 값
        self.hidden_dim = self.hidden_size
        self.intermediate_dim = self.intermediate_size
        self.norm_topk_prob = True


config = DummyConfig()

# mlp = Qwen3MoeMLP(config)
# block = Qwen3MoeSparseMoeBlock(config)
# svd_block = SVDQwen3MoeSparseMoeBlock(config)
# svd_block.SVD(block)

In [6]:
from safetensors.torch import load_file
import torch

# 1️⃣ safetensors 불러오기
state_dict = load_file("/workspace/model-00001-of-00016.safetensors")

# 2️⃣ 필요한 key만 추출 (layers.0.mlp 관련)
mlp_keys = {k.split("layers.0.mlp.")[1]: v for k, v in state_dict.items() if k.startswith("model.layers.0.mlp")}



FileNotFoundError: No such file or directory: /workspace/model-00001-of-00016.safetensors

In [15]:
model.model.layers[0].mlp

Qwen3MoeSparseMoeBlock(
  (gate): Linear(in_features=2048, out_features=128, bias=False)
  (experts): ModuleList(
    (0-127): 128 x Qwen3MoeMLP(
      (gate_proj): Linear(in_features=2048, out_features=768, bias=False)
      (up_proj): Linear(in_features=2048, out_features=768, bias=False)
      (down_proj): Linear(in_features=768, out_features=2048, bias=False)
      (act_fn): SiLU()
    )
  )
)

In [12]:
# 3️⃣ QwenSparseMoeBlock 객체 생성
config = DummyConfig()  # QwenSparseMoeBlock에 맞는 config 정의 필요
mlp = Qwen3MoeSparseMoeBlock(config)

# 4️⃣ weight 로드
mlp.load_state_dict(mlp_keys, strict=False)


<All keys matched successfully>

In [13]:

# --- 테스트용 hidden state 생성 ---
batch_size = 2
seq_len = 16
hidden_states = torch.randn(batch_size, seq_len, config.hidden_size)

# --- forward 호출 ---
output, router_logits = mlp(hidden_states)

print("output shape:", output.shape)           # (batch_size, seq_len, hidden_size)
print("router logits shape:", router_logits.shape) # (batch_size * seq_len, num_experts)
# output shape: torch.Size([2, 16, 2048])
# router logits shape: torch.Size([32, 128])

torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048, 768])
torch.Size([768, 2048])
torch.Size([768, 2048])
torch.Size([2048

In [14]:
svd_block = SVDQwen3MoeSparseMoeBlock(config)
print('done')

In [15]:
svd_block.SVD(mlp)

gate_proj의 0/48번째 svd
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
gate_proj의 10/48번째 svd
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape torch.Size([2048, 167])
us.shapetorch.Size([768, 167])
v.shape to

In [16]:

# --- 테스트용 hidden state 생성 ---
batch_size = 2
seq_len = 16
hidden_states = torch.randn(batch_size, seq_len, config.hidden_size)
# hidden_states.to(torch.bfloat16)
svd_block.to(torch.float32)
# --- forward 호출 ---
output1, router_logits1 = svd_block(hidden_states)

print("output shape:", output1.shape)           # (batch_size, seq_len, hidden_size)
print("router logits shape:", router_logits1.shape) # (batch_size * seq_len, num_experts)
# output shape: torch.Size([2, 16, 2048])
# router logits shape: torch.Size([32, 128])

us1: torch.Size([768, 167])
us2: torch.Size([768, 167])
us3: torch.Size([2048, 167])
v1: torch.Size([167, 2048])
v2: torch.Size([167, 2048])
v3: torch.Size([167, 768])
us1: torch.Size([768, 167])
us2: torch.Size([768, 167])
us3: torch.Size([2048, 167])
v1: torch.Size([167, 2048])
v2: torch.Size([167, 2048])
v3: torch.Size([167, 768])
us1: torch.Size([768, 167])
us2: torch.Size([768, 167])
us3: torch.Size([2048, 167])
v1: torch.Size([167, 2048])
v2: torch.Size([167, 2048])
v3: torch.Size([167, 768])
us1: torch.Size([768, 167])
us2: torch.Size([768, 167])
us3: torch.Size([2048, 167])
v1: torch.Size([167, 2048])
v2: torch.Size([167, 2048])
v3: torch.Size([167, 768])
us1: torch.Size([768, 167])
us2: torch.Size([768, 167])
us3: torch.Size([2048, 167])
v1: torch.Size([167, 2048])
v2: torch.Size([167, 2048])
v3: torch.Size([167, 768])
us1: torch.Size([768, 167])
us2: torch.Size([768, 167])
us3: torch.Size([2048, 167])
v1: torch.Size([167, 2048])
v2: torch.Size([167, 2048])
v3: torch.Size([167

In [19]:
class DynamicSkippingSparseMoeBlockWrapper(nn.Module):
    def __init__(self, model: Qwen3MoeSparseMoeBlock, beta: float):
        super().__init__()
        # assert isinstance(model, Qwen3MoeSparseMoeBlock)
        # assert model.top_k == 2, "Wrapper currently only supports top-2 MoE" #qwen a30은 8
        self.model = model
        self.beta = beta
        self.num_experts = model.num_experts
        self.top_k = model.top_k
        self.hidden_dim = model.gate.in_features
        self.experts = model.experts
        self.gate = model.gate

    def forward(self, hidden_states: torch.Tensor):
        batch_size, seq_len, hidden_dim = hidden_states.shape
        hidden_states_flat = hidden_states.view(-1, hidden_dim)

        # Compute router logits and top-k routing
        router_logits = self.gate(hidden_states_flat)
        routing_weights = F.softmax(router_logits, dim=-1)
        topk_weights, topk_indices = torch.topk(routing_weights, self.top_k, dim=-1)

        # Dynamic skipping: if top-2 < beta * top-1, zero out top-2
        # mask_skip = topk_weights[:, 1] < self.beta * topk_weights[:, 0]
        # topk_weights[mask_skip, 1] = 0
        mask_skip = topk_weights[:, 1:] < self.beta * topk_weights[:, [0]]
        topk_weights[:, 1:][mask_skip] = 0
        topk_weights /= topk_weights.sum(dim=-1, keepdim=True)
        
        topk_weights /= topk_weights.sum(dim=-1, keepdim=True)
        topk_weights = topk_weights.to(hidden_states_flat.dtype)

        # Build expert mask (num_experts, top_k, batch * seq)
        expert_mask = torch.nn.functional.one_hot(topk_indices, num_classes=self.num_experts).permute(2, 1, 0)
        # Apply skip mask to top-2
        expert_mask[:, 1, mask_skip] = 0

        # Prepare output
        final_hidden_states = torch.zeros_like(hidden_states_flat)

        # Compute expert outputs
        for expert_idx in range(self.num_experts):
            idx_topk = torch.where(expert_mask[expert_idx])
            if idx_topk[0].numel() == 0:
                continue

            top_x_list, batch_idx = idx_topk
            current_input = hidden_states_flat[batch_idx]
            expert_output = self.experts[expert_idx](current_input) * topk_weights[batch_idx, top_x_list, None]
            final_hidden_states.index_add_(0, batch_idx, expert_output.to(hidden_states_flat.dtype))

        return final_hidden_states.view(batch_size, seq_len, hidden_dim), router_logits

moe_blk = DynamicSkippingSparseMoeBlockWrapper(svd_block, beta=0.9)

In [20]:

# --- 테스트용 hidden state 생성 ---
batch_size = 2
seq_len = 16
# hidden_states = torch.randn(batch_size, seq_len, config.hidden_size)
# hidden_states.to(torch.bfloat16)
moe_blk.to(torch.float32)
# --- forward 호출 ---
output2, router_logits2 = moe_blk(hidden_states)

print("output shape:", output1.shape)           # (batch_size, seq_len, hidden_size)
print("router logits shape:", router_logits1.shape) # (batch_size * seq_len, num_experts)
# output shape: torch.Size([2, 16, 2048])
# router logits shape: torch.Size([32, 128])

IndexError: too many indices for tensor of dimension 3

In [ ]:
from transformers import AutoModelForCausalLM
from transformers import AutoConfig, AutoModelForCausalLM
from huggingface_hub import login


config = AutoConfig.from_pretrained("Qwen/Qwen3-30B-A3B")  # grabs config only
# model = AutoModelForCausalLM.from_config(config)  # random weights
print('done')

config.json:   0%|          | 0.00/963 [00:00<?, ?B/s]

done


In [7]:
model_name = "Qwen/Qwen3-30B-A3B"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model-00007-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00005-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00003-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

model-00014-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00015-of-00016.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

model-00016-of-00016.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [12]:
output_dir = "./my-awesome-qwen-moe"

# 4. 수정된 모델 저장 (가장 중요한 단계)
# save_pretrained는 모델 가중치, config.json 뿐만 아니라
# 폴더 내의 모든 .py 파일 (custom_moe.py, modeling_qwen2_moe.py 등)을 함께 저장합니다.
model.save_pretrained(output_dir, safe_serialization=True)
tokenizer.save_pretrained(output_dir)

print(f"모델이 '{output_dir}'에 성공적으로 저장되었습니다.")

NameError: name 'tokenizer' is not defined

In [8]:
model.dtype

torch.bfloat16

In [10]:
# with open(expert_freq_path, 'r') as f:
#     expert_freq = json.load(f)
# svd_scale = torch.load(svd_scale_path, map_location='cpu')
# fisher_info = torch.load(fisher_path, map_location="cpu")

for i in tqdm(range(len(model.model.layers)), desc="Merging layers"):
    if i % 5 == 0:
        before_mem = torch.cuda.memory_allocated() / 1e9
        print(f"Layer {i} - Before: {before_mem:.2f}GB")
    
    # 기존 MLP 백업
    old_mlp = model.model.layers[i].mlp
    device = old_mlp.gate.weight.device
    
    # 새 MoE 블록 생성
    Merge_MoE_Block = SVDQwen3MoeSparseMoeBlock(model.config).to(device)
    Merge_MoE_Block.SVD(old_mlp)
    
    # 교체 전 기존 MLP 명시적 삭제
    model.model.layers[i].mlp = None  # 참조 끊기
    del old_mlp                       # 명시적 삭제
    torch.cuda.empty_cache()          # 캐시 정리
    
    # 새 MLP 할당
    model.model.layers[i].mlp = Merge_MoE_Block
    del Merge_MoE_Block
    
    # 메모리 정리
    torch.cuda.empty_cache()
    
    if i % 5 == 0:
        after_mem = torch.cuda.memory_allocated() / 1e9
        print(f"Layer {i} - After: {after_mem:.2f}GB, Saved: {before_mem-after_mem:.2f}GB")


Merging layers:   0%|          | 0/48 [00:00<?, ?it/s]

Layer 0 - Before: 61.06GB
gate_proj의 0/128번째 expert svd
gate_proj의 10/128번째 expert svd
gate_proj의 20/128번째 expert svd
gate_proj의 30/128번째 expert svd
gate_proj의 40/128번째 expert svd
gate_proj의 50/128번째 expert svd
gate_proj의 60/128번째 expert svd
gate_proj의 70/128번째 expert svd
gate_proj의 80/128번째 expert svd
gate_proj의 90/128번째 expert svd
gate_proj의 100/128번째 expert svd
gate_proj의 110/128번째 expert svd
gate_proj의 120/128번째 expert svd
up proj의 0/128번째 svd
up proj의 10/128번째 svd
up proj의 20/128번째 svd
up proj의 30/128번째 svd
up proj의 40/128번째 svd
up proj의 50/128번째 svd
up proj의 60/128번째 svd
up proj의 70/128번째 svd
up proj의 80/128번째 svd
up proj의 90/128번째 svd
up proj의 100/128번째 svd
up proj의 110/128번째 svd
up proj의 120/128번째 svd
down_proj의 0/128번째 svd
down_proj의 10/128번째 svd
down_proj의 20/128번째 svd
down_proj의 30/128번째 svd
down_proj의 40/128번째 svd
down_proj의 50/128번째 svd
down_proj의 60/128번째 svd
down_proj의 70/128번째 svd
down_proj의 80/128번째 svd
down_proj의 90/128번째 svd
down_proj의 100/128번째 svd
down_proj의 110/12

Merging layers:   2%|▏         | 1/48 [00:06<04:52,  6.23s/it]

Layer 0 - After: 60.60GB, Saved: 0.47GB
gate_proj의 0/128번째 expert svd
gate_proj의 10/128번째 expert svd
gate_proj의 20/128번째 expert svd
gate_proj의 30/128번째 expert svd
gate_proj의 40/128번째 expert svd
gate_proj의 50/128번째 expert svd
gate_proj의 60/128번째 expert svd
gate_proj의 70/128번째 expert svd
gate_proj의 80/128번째 expert svd
gate_proj의 90/128번째 expert svd
gate_proj의 100/128번째 expert svd
gate_proj의 110/128번째 expert svd
gate_proj의 120/128번째 expert svd
up proj의 0/128번째 svd
up proj의 10/128번째 svd
up proj의 20/128번째 svd
up proj의 30/128번째 svd
up proj의 40/128번째 svd
up proj의 50/128번째 svd
up proj의 60/128번째 svd
up proj의 70/128번째 svd
up proj의 80/128번째 svd
up proj의 90/128번째 svd
up proj의 100/128번째 svd
up proj의 110/128번째 svd
up proj의 120/128번째 svd
down_proj의 0/128번째 svd
down_proj의 10/128번째 svd
down_proj의 20/128번째 svd
down_proj의 30/128번째 svd
down_proj의 40/128번째 svd
down_proj의 50/128번째 svd
down_proj의 60/128번째 svd
down_proj의 70/128번째 svd
down_proj의 80/128번째 svd
down_proj의 90/128번째 svd
down_proj의 100/128번째 svd
dow

Merging layers:   4%|▍         | 2/48 [00:12<04:44,  6.18s/it]

...

down_proj의 120/128번째 svd
Layer 45 - After: 38.83GB, Saved: 0.48GB
gate_proj의 0/128번째 expert svd
gate_proj의 10/128번째 expert svd
gate_proj의 20/128번째 expert svd
gate_proj의 30/128번째 expert svd
gate_proj의 40/128번째 expert svd
gate_proj의 50/128번째 expert svd
gate_proj의 60/128번째 expert svd
gate_proj의 70/128번째 expert svd
gate_proj의 80/128번째 expert svd
gate_proj의 90/128번째 expert svd
gate_proj의 100/128번째 expert svd
gate_proj의 110/128번째 expert svd
gate_proj의 120/128번째 expert svd
up proj의 0/128번째 svd
up proj의 10/128번째 svd
up proj의 20/128번째 svd
up proj의 30/128번째 svd
up proj의 40/128번째 svd
up proj의 50/128번째 svd
up proj의 60/128번째 svd
up proj의 70/128번째 svd
up proj의 80/128번째 svd
up proj의 90/128번째 svd
up proj의 100/128번째 svd
up proj의 110/128번째 svd
up proj의 120/128번째 svd
down_proj의 0/128번째 svd
down_proj의 10/128번째 svd
down_proj의 20/128번째 svd
down_proj의 30/128번째 svd
down_proj의 40/128번째 svd
down_proj의 50/128번째 svd
down_proj의 60/128번째 svd
down_proj의 70/128번째 svd
down_proj의 80/128번째 svd
down_proj의 90/128번째 svd
do

Merging layers:  98%|█████████▊| 47/48 [04:48<00:06,  6.22s/it]

down_proj의 120/128번째 svd
gate_proj의 0/128번째 expert svd
gate_proj의 10/128번째 expert svd
gate_proj의 20/128번째 expert svd
gate_proj의 30/128번째 expert svd
gate_proj의 40/128번째 expert svd
gate_proj의 50/128번째 expert svd
gate_proj의 60/128번째 expert svd
gate_proj의 70/128번째 expert svd
gate_proj의 80/128번째 expert svd
gate_proj의 90/128번째 expert svd
gate_proj의 100/128번째 expert svd
gate_proj의 110/128번째 expert svd
gate_proj의 120/128번째 expert svd
up proj의 0/128번째 svd
up proj의 10/128번째 svd
up proj의 20/128번째 svd
up proj의 30/128번째 svd
up proj의 40/128번째 svd
up proj의 50/128번째 svd
up proj의 60/128번째 svd
up proj의 70/128번째 svd
up proj의 80/128번째 svd
up proj의 90/128번째 svd
up proj의 100/128번째 svd
up proj의 110/128번째 svd
up proj의 120/128번째 svd
down_proj의 0/128번째 svd
down_proj의 10/128번째 svd
down_proj의 20/128번째 svd
down_proj의 30/128번째 svd
down_proj의 40/128번째 svd
down_proj의 50/128번째 svd
down_proj의 60/128번째 svd
down_proj의 70/128번째 svd
down_proj의 80/128번째 svd
down_proj의 90/128번째 svd
down_proj의 100/128번째 svd
down_proj의 110/128

Merging layers: 100%|██████████| 48/48 [04:54<00:00,  6.14s/it]

down_proj의 120/128번째 svd


In [9]:
torch.cuda.memory_allocated() / 1e9

61.064270336

In [11]:
torch.cuda.memory_allocated() / 1e9

37.858816512

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
model_inputs

In [ ]:

print('start')
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 
print('out gen')
# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
print('decoded')
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

start


In [ ]:
print('')

In [33]:
torch.cuda.empty_cache()


In [ ]:
model = model.half()

In [26]:
model.model.layers[i].mlp.experts[i].gate_proj.weight.dtype

torch.bfloat16

In [22]:
model.dtype

torch.bfloat16

In [10]:
print("===== Model Named Parameters =====")
for name, param in model.named_parameters():
    # param.requires_grad는 해당 파라미터가 학습 대상인지 여부를 알려줍니다.
    print(f"Name: {name}, Shape: {param.shape}")

===== Model Named Parameters =====
Name: model.embed_tokens.weight, Shape: torch.Size([151936, 2048])
Name: model.layers.0.self_attn.q_proj.weight, Shape: torch.Size([4096, 2048])
Name: model.layers.0.self_attn.k_proj.weight, Shape: torch.Size([512, 2048])
Name: model.layers.0.self_attn.v_proj.weight, Shape: torch.Size([512, 2048])
Name: model.layers.0.self_attn.o_proj.weight, Shape: torch.Size([2048, 4096])
Name: model.layers.0.self_attn.q_norm.weight, Shape: torch.Size([128])
Name: model.layers.0.self_attn.k_norm.weight, Shape: torch.Size([128])
Name: model.layers.0.mlp.gate.weight, Shape: torch.Size([128, 2048])
Name: model.layers.0.mlp.experts_v1_shared_gate.weight, Shape: torch.Size([167, 2048])
Name: model.layers.0.mlp.experts_v2_shared_up.weight, Shape: torch.Size([167, 2048])
Name: model.layers.0.mlp.experts_v3_shared_down.weight, Shape: torch.Size([167, 768])
Name: model.layers.0.mlp.experts.0.v1.weight, Shape: torch.Size([167, 2048])
Name: model.layers.0.mlp.experts.0.us1.wei

In [ ]:
from huggingface_hub import login

!git lfs install
!git clone https://huggingface.co/veritasium/my_model


git: 'lfs' is not a git command. See 'git --help'.

The most similar command is
	log
Cloning into 'my_model'...
remote: Enumerating objects: 3, done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 3 (from 1)
Unpacking objects: 100% (3/3), 1.05 KiB | 23.00 KiB/s, done.
[Errno 2] No such file or directory: 'your-model-name'
/workspace
fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: not a git repository (or any parent up to mount point /)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [ ]:
# %cd my_model
# !mv /workspace/my-awesome-qwen-moe/* .
!git add .
!git commit -m "Upload custom model"
!git push

In [ ]:
repo.git_add()
repo.git_commit("Upload custom model")
repo.git_push()

In [ ]:
!pip install huggingface_hub -q
from huggingface_hub import login



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:
from huggingface_hub import login



In [2]:
print('a')

a


In [ ]:
print('a')
!hf upload veritasium/my_model /workspace/my_model/model-00007-of-00008.safetensors

a
Processing Files (0 / 0)      : |                  |  0.00B /  0.00B            
New Data Upload               : |                  |  0.00B /  0.00B            

  ...0007-of-00008.safetensors:   0%|              |  556kB / 5.00GB            

Processing Files (0 / 1)      :   0%|              |  556kB / 5.00GB,  185kB/s  
New Data Upload               :   1%|              |  556kB / 67.1MB,  185kB/s  

  ...0007-of-00008.safetensors:   0%|              |  556kB / 5.00GB            

Processing Files (0 / 1)      :   0%|              | 1.11MB / 5.00GB,  327kB/s  
New Data Upload               :   2%|▏             | 1.11MB / 67.1MB,  327kB/s  

  ...0007-of-00008.safetensors:   0%|              | 1.11MB / 5.00GB            

Processing Files (0 / 1)      :   0%|              | 1.67MB / 5.00GB,  439kB/s  
New Data Upload               :   2%|▎             | 1.67MB / 67.1MB,  439kB/s  

  ...0007-of-00008.safetensors:   0%|              | 1.67MB / 5.00GB            

  ...0007-of-00008

In [3]:
!pip install hf_transfer -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 37.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
!export HF_HUB_ENABLE_HF_TRANSFER=1
print('done')

done


In [17]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"


In [2]:

# # Install the Hugging Face CLI
!pip install -U "huggingface_hub[cli]"

# # Login with your Hugging Face credentials
# !hf auth loginhf_rYgfMxvIAcWqpVVubfLlvQRgRPnbdldBIi


# # Push your model files
!hf upload veritasium/my_model . 


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [ ]:

!hf upload veritasium/my_model . 

In [1]:
%cd my_model

/workspace/my_model
